In [ ]:
!export MAKEFLAGS="-j$(nproc)"
!pip install diffusers transformers accelerate xformers huggingface_hub[hf_transfer] hf_transfer \
    pillow insightface opencv-python apex gradio onnxruntime-gpu timm pickleshare \
    SentencePiece ftfy einops facexlib fire onnx bencodepy torrentp ninja
!pip install git+https://github.com/XPixelGroup/BasicSR
!pip install -U git+https://github.com/qubvel/transformers@fix-custom-kernels
!pip install flash-attn

In [ ]:
%%capture
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfaHZqck9VTXFvTXF3dW9HR3JoTlZKSWlsZUtFTlNQbXRjTw==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_HF_TRANSFER=1

In [ ]:
!pip install lmdeploy timm


In [ ]:
from lmdeploy.vl import load_image

load_image('/workspace/dtback/dataset_creation/data/dataset/0/0')


In [ ]:
!export MAKEFLAGS="-j$(nproc)"
!pip install transformers huggingface_hub[hf_transfer] hf_transfer pillow torch decord
!pip install lmdeploy timm
!pip install flash-attn
!huggingface-cli download OpenGVLab/InternVL2_5-26B-MPO-AWQ # for dataset labeling

In [ ]:
%cd dataset_creation
!python3 label.py
%cd ..

```bash
sudo -v ; curl https://rclone.org/install.sh | sudo bash
sudo apt-get update
apt-get update && apt-get install -y fuse3 -y
```
-- write rclone.conf --
```bash
mkdir -p /workspace/dtback/dataset_creation/data/dataset
cd /workspace/dtback/dataset_creation/data/dataset
rclone sync drive:dataset/stage_0 . --progress # syncying stage 0
for tar in *.tar; do
    echo "Extracting $tar..."
    tar -xf "$tar" && rm "$tar"
done
```

```bash
sudo apt update && sudo apt install screen -y
cd /workspace/dtback/dataset_creation
screen -S gather
python3 gather.py > gather.log 2>&1
python3 label.py > label.log 2>&1
# ctrl+a d
screen -r gather
```

In [ ]:
!huggingface-cli download black-forest-labs/FLUX.1-dev
!huggingface-cli download OpenGVLab/InternViT-300M-448px-V2_5
!huggingface-cli download IDEA-Research/grounding-dino-tiny
# !huggingface-cli download OpenGVLab/InternVL2_5-26B-MPO-AWQ # for dataset labeling

In [ ]:
# upload everything

import os, asyncio, base64, shutil, random, string, logging, sys, requests, glob, cv2, re
import insightface, torch, pickle, time, bencodepy, hashlib,  json, subprocess, multiprocessing

import nest_asyncio
nest_asyncio.apply()

async def upload_bundle(bundle, local, remote):
    archive = f"{bundle}.tar"
    proc = await asyncio.create_subprocess_exec(
        "tar", "-cf", archive, "-C", local, bundle,
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    await proc.communicate()
    proc = await asyncio.create_subprocess_exec(
        "rclone", "copy", archive, remote,
        "--checksum", "--transfers=64", "--checkers=64",
        "--fast-list", "--multi-thread-streams=4", "--progress",
        stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE
    )
    await proc.communicate()
    os.remove(archive)
    logging.info(f"Processed and uploaded bundle {bundle}")


async def upload_to_drive(stage=0):
    local = "data/dataset"
    remote = f"drive:dataset/stage_{stage}"
    bundles = [d for d in os.listdir(local) if os.path.isdir(os.path.join(local, d)) and d.isdigit()]
    if bundles:
        await asyncio.gather(*(upload_bundle(bundle, local, remote) for bundle in bundles))


async def sync_loop():
    while True:
        await upload_to_drive(0)
        await asyncio.sleep(12*60)